In [0]:
import pdb
import argparse
import numpy as np
from tqdm import tqdm
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import MultiStepLR

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from torchvision import datasets, transforms

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.

    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img


In [0]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                               padding=0, bias=False) or None
    def forward(self, x):
        if not self.equalInOut:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)

class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate)
    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, dropRate):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)

class WideResNet(nn.Module):
    def __init__(self, depth, num_classes, widen_factor=1, dropRate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert((depth - 4) % 6 == 0)
        n = (depth - 4) / 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, dropRate)
        # 2nd block
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, dropRate)
        # 3rd block
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, dropRate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(nChannels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))

        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        out = self.fc(out)
        return out


In [0]:
normalize = transforms.Normalize(mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x / 255.0 for x in [63.0, 62.1, 66.7]])

In [0]:
train_transform = transforms.Compose([])
train_transform.transforms.append(transforms.RandomCrop(32, padding=4))
train_transform.transforms.append(transforms.RandomHorizontalFlip())
train_transform.transforms.append(transforms.ToTensor())
train_transform.transforms.append(normalize)
train_transform.transforms.append(Cutout(n_holes=1, length=16))

In [0]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize])

In [8]:
trainset = torchvision.datasets.CIFAR100(root='./data',train=True,download=True,transform=train_transform)

Files already downloaded and verified


In [0]:
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True,num_workers=2,pin_memory=True)

In [10]:
testset = torchvision.datasets.CIFAR100(root='./data',train=False,download=True,transform = test_transform)

Files already downloaded and verified


In [0]:
testLoader = torch.utils.data.DataLoader(testset, batch_size=64,shuffle=False,num_workers=2,pin_memory=True)

In [0]:
classes = tuple(trainset.classes)

In [0]:
cnn = WideResNet(28,100)

In [0]:
path_wcuda = F"/content/gdrive/My Drive/checkpoints/cudamodel2cifar100.pt"

In [15]:
cnn.load_state_dict(torch.load(path_wcuda))
cnn.eval()

WideResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (block1): NetworkBlock(
    (layer): Sequential(
      (0): BasicBlock(
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (1): BasicBlock(
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
      

In [0]:
import torch.optim as optim
cnn = cnn.cuda()
criterion = nn.CrossEntropyLoss().cuda()
cnn_optimizer = torch.optim.SGD(cnn.parameters(), lr=0.1,
                                momentum=0.9, nesterov=True, weight_decay=5e-4)

In [0]:
def test(loader):
    cnn.eval()    # Change model to 'eval' mode (BN uses moving mean/var).
    correct = 0.
    total = 0.
    for images, labels in loader:
        images = images.cuda()
        labels = labels.cuda()

        with torch.no_grad():
            pred = cnn(images)

        pred = torch.max(pred.data, 1)[1]
        total += labels.size(0)
        correct += (pred == labels).sum().item()

    val_acc = correct / total
    cnn.train()
    return val_acc

In [18]:
test_acc = test(testLoader)
print(test_acc)

0.7014


In [0]:
def train(sepoch, eepoch, lr):
  for param_group in cnn_optimizer.param_groups:
        param_group['lr'] = lr

  for epoch in range(sepoch,eepoch):

    xentropy_loss_avg = 0.
    correct = 0.
    total = 0.

    progress_bar = tqdm(trainLoader)
    for i, (images, labels) in enumerate(progress_bar):
        progress_bar.set_description('Epoch ' + str(epoch))

        images = images.cuda()
        labels = labels.cuda()

        cnn.zero_grad()
        pred = cnn(images)

        xentropy_loss = criterion(pred, labels)
        xentropy_loss.backward()
        cnn_optimizer.step()

        xentropy_loss_avg += xentropy_loss.item()

        # Calculate running average of accuracy
        pred = torch.max(pred.data, 1)[1]
        total += labels.size(0)
        correct += (pred == labels.data).sum().item()
        accuracy = correct / total

        progress_bar.set_postfix(
            xentropy='%.3f' % (xentropy_loss_avg / (i + 1)),
            acc='%.3f' % accuracy)

    test_acc = test(testLoader)
    tqdm.write('test_acc: %.3f' % (test_acc))

In [35]:
train(0,50,0.001)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.697


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.698


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.693


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.695


Epoch 49: 100%|██████████| 391/391 [00:23<00:00, 16.53it/s, acc=0.798, xentropy=0.676]


test_acc: 0.692


In [36]:
train(50,75,0.0007)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.703


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.705


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.699


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.703


Epoch 74: 100%|██████████| 391/391 [00:23<00:00, 16.30it/s, acc=0.814, xentropy=0.633]


test_acc: 0.699


In [37]:
train(50,75,0.0005)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.708


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.705


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.704


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.706


Epoch 74: 100%|██████████| 391/391 [00:24<00:00, 16.24it/s, acc=0.823, xentropy=0.604]


test_acc: 0.706


In [38]:
train(100,125,0.00025)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.710


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.709


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.710


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


Epoch 124: 100%|██████████| 391/391 [00:24<00:00, 16.14it/s, acc=0.835, xentropy=0.560]


test_acc: 0.711


In [39]:
train(125,150,0.0001)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.711


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.714


Epoch 149: 100%|██████████| 391/391 [00:24<00:00, 16.26it/s, acc=0.843, xentropy=0.536]


test_acc: 0.714


In [40]:
train(150,175,0.00005)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.711


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.710


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.710


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


Epoch 174: 100%|██████████| 391/391 [00:24<00:00, 16.11it/s, acc=0.845, xentropy=0.533]


test_acc: 0.714


In [41]:
train(150,175,0.000025)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.714


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.714


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


Epoch 174: 100%|██████████| 391/391 [00:24<00:00, 16.17it/s, acc=0.850, xentropy=0.518]


test_acc: 0.712


In [42]:
train(175,200,0.000001)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.714


Epoch 199: 100%|██████████| 391/391 [00:24<00:00, 16.28it/s, acc=0.850, xentropy=0.519]


test_acc: 0.713


In [43]:
train(200,225,0.00001)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.711


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.713


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


Epoch 224: 100%|██████████| 391/391 [00:23<00:00, 16.44it/s, acc=0.850, xentropy=0.517]


test_acc: 0.711


In [44]:
train(225,250,0.00005)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.711


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.711


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.711


Epoch 249: 100%|██████████| 391/391 [00:23<00:00, 16.43it/s, acc=0.847, xentropy=0.524]


test_acc: 0.714


In [45]:
train(250,275,0.0001)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.710


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.714


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


Epoch 274: 100%|██████████| 391/391 [00:23<00:00, 16.41it/s, acc=0.847, xentropy=0.529]


test_acc: 0.711


In [0]:
path = F"/content/gdrive/My Drive/checkpoints/cudamodel2cifar100.pt"

In [0]:
torch.save(cnn.cuda().state_dict(), path)

In [20]:
train(250,275,0.0001)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.708


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.709


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.709


Epoch 274: 100%|██████████| 391/391 [00:24<00:00, 16.21it/s, acc=0.847, xentropy=0.526]


test_acc: 0.708


In [21]:
train(275,300,0.00001)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


Epoch 299: 100%|██████████| 391/391 [00:24<00:00, 15.98it/s, acc=0.850, xentropy=0.517]


test_acc: 0.714


In [22]:
train(300,325,0.000007)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.711


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


Epoch 324: 100%|██████████| 391/391 [00:24<00:00, 15.99it/s, acc=0.852, xentropy=0.511]


test_acc: 0.713


In [23]:
train(300,325,0.000002)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.710


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


Epoch 324: 100%|██████████| 391/391 [00:24<00:00, 16.01it/s, acc=0.851, xentropy=0.514]


test_acc: 0.714


In [0]:
path = F"/content/gdrive/My Drive/checkpoints/cudamodel3cifar100.pt"
torch.save(cnn.cuda().state_dict(), path)

In [31]:
list(cnn.cuda().parameters())[0]

Parameter containing:
tensor([[[[-1.3548e-01, -7.9588e-03, -1.8320e-01],
          [-1.6720e-01,  6.5156e-01,  2.4132e-01],
          [-2.1406e-01,  3.6903e-01,  8.9527e-02]],

         [[ 5.9999e-02, -3.8409e-02,  6.2774e-02],
          [-8.2885e-03, -3.5415e-01, -1.7258e-01],
          [ 1.0805e-01, -2.0573e-01, -4.8925e-02]],

         [[ 1.0192e-01,  5.1675e-02,  5.3686e-02],
          [ 8.6815e-02, -2.9622e-01, -5.3761e-02],
          [ 1.3501e-01, -1.9425e-01, -2.7947e-02]]],


        [[[ 7.7349e-02,  1.3315e-01,  1.3910e-01],
          [-5.9730e-02, -3.4876e-01, -2.4625e-01],
          [ 4.0981e-02, -3.9260e-01, -2.1218e-01]],

         [[-3.9853e-02, -1.3246e-02, -4.7400e-02],
          [ 3.1156e-02,  2.0165e-01,  1.2023e-01],
          [-1.1370e-02,  1.5111e-01,  1.4393e-01]],

         [[-1.9266e-02, -1.3493e-01, -1.0535e-01],
          [-3.4208e-03,  1.5750e-01,  7.9211e-02],
          [ 7.9302e-04,  1.3393e-01,  8.8985e-02]]],


        [[[ 1.3319e-02, -9.7780e-02, -3.4708

In [29]:
train(300,325,0.000005)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.714


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.713


Epoch 324: 100%|██████████| 391/391 [00:24<00:00, 16.08it/s, acc=0.851, xentropy=0.514]


test_acc: 0.715


In [32]:
train(325,400,0.000001)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.711


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


Epoch 399: 100%|██████████| 391/391 [00:24<00:00, 15.68it/s, acc=0.851, xentropy=0.512]


test_acc: 0.712


In [33]:
train(325,400,0.000001)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.711


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

...

test_acc: 0.711


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.714


Epoch 399: 100%|██████████| 391/391 [00:24<00:00, 15.74it/s, acc=0.851, xentropy=0.515]


test_acc: 0.713


In [0]:
train(325,400,0.0000005)

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


  0%|          | 0/391 [00:00<?, ?it/s]

...

  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.714


  0%|          | 0/391 [00:00<?, ?it/s]

test_acc: 0.712


Epoch 362:  73%|███████▎  | 285/391 [00:17<00:06, 15.84it/s, acc=0.852, xentropy=0.512]